# ML-Ask を用いたツイートの感情解析 (最終)

## 感情解析ライブラリ ML-Ask
ML-Ask とは中村[1]の提唱した 10種類の感情モデルを基に，感情表現辞典に記載された表現をキーワードとして探索し，感情を推定する手法[2, 3]である。

感情は喜, 怒, 哀, 怖, 恥, 好, 厭, 昂, 安, 驚の10種類に分類される。

ML-Ask に関する概要は以下の Web ページに記載されている。
> http://arakilab.media.eng.hokudai.ac.jp/~ptaszynski/repository/mlask.htm

[1] 中村明, 感情表現辞典, 東京堂出版, 1993.

[2] Michal Ptaszynski, Pawel Dybala, Rafal Rzepka and Kenji Araki, “Affecting Corpora: Experiments with Automatic Affect Annotation System - A Case Study of the 2channel Forum -”, In Proceedings of The Conference of the Pacific Association for Computational Linguistics (PACLING-09), September 1-4, 2009, Hokkaido University, Sapporo, Japan, pp. 223-228.

[3] Michal Ptaszynski, Pawel Dybala, Wenhan Shi, Rafal Rzepka and Kenji Araki, “A System for Affect Analysis of Utterances in Japanese Supported with Web Mining”, Journal of Japan Society for Fuzzy Theory and Intelligent Informatics, Vol. 21, No. 2 (April), pp. 30-49 (194-213), 2009.

### pymlask のインストール
ML-Ask を Python で実装したライブラリが pymlask である。

https://github.com/ikegami-yukino/pymlask?files=1

Anaconda3 から Anaconda Prompt を起動し，以下の pip コマンドを入力すると pymlask がインストールされる。（ここで先頭の "$" はプロンプトである)

`$ pip install pymlask`

なお，ML-Ask の動作には MeCab が必要となる。（前回の実験でインストール済みのはず）

### Plotly Express のインストール (可視化の準備)
Anaconda Prompt にて以下の pip コマンドを入力すること

`$ pip install plotly_express`

Plotly Express の詳細は下記のサイトを参照のこと：
- https://plotly.com/python/plotly-express/
- https://qiita.com/hanon/items/d8cbe25aa8f3a9347b0b

### ML-Ask による感情推定(1)
以下は感情をひとつだけ含む例である。「美味しくない」に対して「厭」の感情が紐付けられている。

また，「美味しい」＋「ない」から negation も考慮されていることが分かる(ML-Ask では Contextual Valence Shifters (CVS)により negation および intensifier を判定する)。

(参考) ML-Ask では感情の他に極性(orinetation)，態(activation; 能動的な受動的か)も判定する。
以下の例では極性が "NEGATIVE", 態が "NEUTRAL" と判定されている。

In [8]:
from mlask import MLAsk
emotion_analyzer = MLAsk()
result = emotion_analyzer.analyze("バナナは美味しくない")
result

{'text': 'バナナは美味しくない',
 'emotion': defaultdict(list, {'iya': ['美味しい*CVS']}),
 'orientation': 'NEGATIVE',
 'activation': 'NEUTRAL',
 'emoticon': None,
 'intension': 0,
 'intensifier': {},
 'representative': ('iya', ['美味しい*CVS'])}

### ML-Ask による感情推定(2)
以下はひとつのテキストに複数の感情を含む例である。
ここでは「心配」，「楽しみ」がそれぞれ「怖」「喜」に対応する感情語として抽出されている。
また，代表的な感情(representative)として「喜」が推定される。

なお，極性は "NEUTRAL"，態は "ACTIVE" である。

In [18]:
text = "コロナウイルスの広がりが心配だ。でも，明日のボーナスが楽しみ！"
result = emotion_analyzer.analyze(text)
result

{'text': 'コロナウイルスの広がりが心配だ。でも，明日のボーナスが楽しみ！',
 'emotion': defaultdict(list, {'kowa': ['心配'], 'yorokobi': ['楽しみ']}),
 'orientation': 'NEUTRAL',
 'activation': 'ACTIVE',
 'emoticon': None,
 'intension': 2,
 'intensifier': {'interjections': ['ウイ'], 'exclamation': ['！']},
 'representative': ('yorokobi', ['楽しみ'])}

# 以上，ここまで説明済み！
これ以降が最終回の内容です。

## コロナに言及したツイート数
NHK「特設サイト 新型コロナウイルス」に掲載されている時系列ニュース

https://www3.nhk.or.jp/news/special/coronavirus/chronology/

によると，2020年1月6日の「中国 武漢で原因不明の肺炎 厚労省が注意喚起」から新型コロナウイルスに関するニュースが始まっている。

そこで，2020年1月6日〜8月31日の期間について，コロナウイルスに言及したツイートの投稿数を見てみよう。

ここで，Twitter利用者の感情解析が目的であることから，リツイートおよび URL を含むツイート（ニュースの転載と思われる）を排除した。また，メンションは個々人の感情の発露を反映すると仮定し，カウントする。

＃ いくつかデータ取得できなかった日がある。ご勘弁ください。

In [20]:
import datetime
import sys

'''
データを収集する期間を指定
dates = ["20200401", "20200402", ...., "20200630"] というリストを作る
'''
def set_period(from_year, from_month, from_day, until_year, until_month, until_day):
    dates = list()

    d = datetime.datetime(from_year, from_month, from_day)
    while(True):
        if d > datetime.datetime(until_year, until_month, until_day): break
        dates.append("{}{:02d}{:02d}".format(d.year, d.month, d.day))
        d += datetime.timedelta(days=1)
    
    return dates

if __name__ == '__main__':
    # 開始時刻をメモ
    start_time = datetime.datetime.now()
    
    results = dict()

    # データ収集期間の指定
    dates = set_period(2020, 1, 6, 2020, 8, 31)

    # 各月日の感情を推定 ＆ カウントアップ
    for d in dates:
        # ファイル名，特にパスは各自の環境に合わせて！
        # filename = "/home/muto/Dropbox/ExpSSE3/Tweets/{}_MERS.txt".format(d) # for Linux
        filename = r"C:\Users\yoshi\Dropbox\ExpSSE3\Tweets\{}_MERS.txt".format(d) # for Windows
        
        # ツイート数のカウント
        # (メモ) 存在しないファイルのため try / except
        try:
            results[d] = sum([1 for _ in open(filename, encoding="utf8")])
        except:
            results[d] = None
    
    # pandas データフレームに値を押し込む
    # 理由１：Plotly Express で処理するため
    # 理由２：感情を日本語で表現するため
    import pandas as pd
    df = pd.DataFrame(columns=['date', 'count'])
    
    #import matplotlib.pyplot as plt
    #from matplotlib.font_manager import FontProperties
    #fp = FontProperties(fname=r'/usr/share/fonts/truetype/fonts-japanese-gothic.ttf', size=16)

    for d in dates:
        dt = datetime.datetime(int(d[0:4]), int(d[4:6]), int(d[6:8]))
        if results[d] is not None:
            df = df.append(pd.DataFrame([[dt, results[d]]],
                                         columns=['date', 'count']))
    
    df = df.reset_index()
    
    # print(df)

    # Plotly Express を用いた可視化
    import plotly_express as px
    # df_melt = df.melt(id_vars='date', value_vars=j_emotions.values()) # value_vars=emotions)
    fig = px.bar(df, x="date", y="count", log_y=False, title="# of tweets that mention Corona virus")
    fig.show()


## (A) 特定の感情のみに絞り込んだ解析
ML-Ask では 厭(iya), 驚(odoroki), 安(yasu), 喜(yorokobi), 怖(kowa), 好(suki), 昂(takaburi), 怒(ikari), 哀(aware), 恥(haji)の10種類の感情を定義するが，COVID-19 に言及したツイートの分析という観点から
**「厭(iya), 安(yasu), 喜(yorokobi), 怖(kowa), 怒(ikari)」** の５つの感情のみ対象とする。

(注意) 以下のコードを実行する場合，３〜４時間を平気で要する。故に，予め１〜８月の月単位で上記５感情に対するツイート割合を求めている（後記の (B)）から，これらを読み込むコード（後記の (C)）を使う方がよい。

→ 感情の種類を増やして解析したい場合，下記コードをご利用ください。

### 最初にモジュールのインポート，感情推定の関数等の定義を行う

In [6]:
# ML-Ask の出力する結果のうち representative を用いた結果
from mlask import MLAsk
import datetime
import sys

# 感情推定
def emotion_estimation(filename, emotions, verb=False):
    import os
    if os.path.exists(filename) is False:
        return []
    
    # ML-Ask コンストラクタ
    emotion_analyzer = MLAsk()

    # 感情の数を収める辞書の初期化
    emotion_counter = dict()
    for emotion in emotions:
        emotion_counter[emotion] = 0

    # 感情を含むツイートの総数をカウント
    emotional_tweet_counter = 0

    # ツイートの総数をカウント
    whole_tweet_counter = 0
    
    '''
    ファイルの形式は
    status_id \t retweet_count \t favorite_coun \t text
    '''
    with open(filename, "r", encoding='utf-8') as f:
        for line in f.readlines():
            # 何故か分からないが，(id, retweet_count 等が含まれないテキストのみの行が存在する
            try:
                status_id, retweet_count, favorite_count, text = line.rstrip('\n').split('\t')
            except ValueError:
                continue
         
            whole_tweet_counter += 1
            
            status_id = int(status_id)
            retweet_count = int(retweet_count)
            favorite_count = int(favorite_count)
            
            # URL を含むツイートを排除（超簡易版の実装）
            if "http" in text:
                continue

            #if text[0] == '@':
            #    continue
                
            # 感情解析
            result = emotion_analyzer.analyze(text)
            if result['emotion'] is not None:
                activation = result['activation']
                if activation == 'ACTIVE' or activation == 'PASSIVE':
                    if 'representative' in result:
                        emotion = result['representative'][0] # 代表的な感情のみ取り出す
                        if emotion in emotions:
                            emotional_tweet_counter += 1 # 感情を含むツイート数
                            emotion_counter[emotion] += 1
                            '''
                            if favorite_count == 0:
                                emotion_counter[emotion] += 1
                            else:
                                emotion_counter[emotion] += (1 + favorite_count)
                            '''

                            # デバッグ用
                            if verb == True:
                                print("--------------------")
                                print(text)
                                print(emotion)

    # 感情カウント値の正規化
    if emotional_tweet_counter > 0:
        for key in emotion_counter:
            # emotion_counter[key] /= emotional_tweet_counter
            emotion_counter[key] /= whole_tweet_counter
        
    return emotion_counter

'''
データを収集する期間を指定
dates = ["20200401", "20200402", ...., "20200630"] というリストを作る
'''
def set_period(from_year, from_month, from_day, until_year, until_month, until_day):
    dates = list()

    d = datetime.datetime(from_year, from_month, from_day)
    while(True):
        if d > datetime.datetime(until_year, until_month, until_day): break
        dates.append("{}{:02d}{:02d}".format(d.year, d.month, d.day))
        d += datetime.timedelta(days=1)
    
    return dates



### メイン関数を以下に示す。

In [8]:
if __name__ == '__main__':
    # 開始時刻をメモ
    start_time = datetime.datetime.now()
    
    results = dict()

    # ML-Ask にて定義される感情の一覧
    # 可視化の際，日本語を用いるため ML-Ask の出力との対応表も準備する
    emotions = ['iya', 'yasu', 'yorokobi', 'kowa', 'ikari']
    j_emotions = {'iya':'厭', 'yasu':'安', 'yorokobi':'喜', 'kowa':'怖', 'ikari':'怒'}

    # データ収集期間の指定
    dates = set_period(2020, 1, 6, 2020, 1, 31) # ここの期間を適宜変更する

    # 各月日の感情を推定 ＆ カウントアップ
    for d in dates:
        # ファイル名，特にパスは各自の環境に合わせて！
        # filename = "/home/muto/Dropbox/ExpSSE3/Tweets/{}_MERS.txt".format(d) # for Linux
        filename = r"C:\Users\yoshi\Dropbox\ExpSSE3\Tweets\{}_MERS.txt".format(d) # for Windows
        
        # 感情推定
        emotion_counter = emotion_estimation(filename, emotions)
        print(d, emotion_counter, file=sys.stderr)
        
        # データを取れていない日に対処するための措置
        if emotion_counter == []: continue

        # 月日 "d" の感情カウントを保存
        results[d] = emotion_counter
    
    # 終了時刻をメモ
    end_time = datetime.datetime.now()
    
    print("処理時間 {}".format(end_time - start_time), file=sys.stderr)
    
    # pandas データフレームに値を押し込む
    # 理由１：Plotly Express で処理するため
    # 理由２：感情を日本語で表現するため
    import pandas as pd
    df = pd.DataFrame(columns=['date']+emotions)
    
    #import matplotlib.pyplot as plt
    #from matplotlib.font_manager import FontProperties
    #fp = FontProperties(fname=r'/usr/share/fonts/truetype/fonts-japanese-gothic.ttf', size=16)

    for d in dates:
        dt = datetime.datetime(int(d[0:4]), int(d[4:6]), int(d[6:8]))
        try:
            df = df.append(pd.DataFrame([[dt, results[d]['iya'], results[d]['yasu'],
                                          results[d]['yorokobi'], results[d]['kowa'], results[d]['ikari']]],
                                         columns=['date']+emotions))
        except:
            pass # データを取れていない日に対処するための措置
    
    # 列名をローマ字表記から日本語表記へリネーム
    df = df.rename(columns=j_emotions)
    
    df = df.reset_index()
    
    print(df)

    # Plotly Express を用いた可視化
    import plotly_express as px
    df_melt = df.melt(id_vars='date', value_vars=j_emotions.values()) # value_vars=emotions)
    fig = px.line(df_melt, x="date", y="value", color='variable',
                  title="Emotions ....")
    fig.show()

20200106 {'iya': 0.0, 'yasu': 0.0, 'yorokobi': 0.0, 'kowa': 0.08333333333333333, 'ikari': 0.03333333333333333}
20200107 {'iya': 0.0, 'yasu': 0.0, 'yorokobi': 0.0, 'kowa': 0.09302325581395349, 'ikari': 0.0}
20200108 {'iya': 0.0, 'yasu': 0.0, 'yorokobi': 0.0, 'kowa': 0.038461538461538464, 'ikari': 0.0}
20200109 {'iya': 0.000851063829787234, 'yasu': 0.006808510638297872, 'yorokobi': 0.000851063829787234, 'kowa': 0.03234042553191489, 'ikari': 0.000851063829787234}
20200110 []
20200111 {'iya': 0.00267379679144385, 'yasu': 0.0053475935828877, 'yorokobi': 0.0, 'kowa': 0.06417112299465241, 'ikari': 0.0}
20200112 {'iya': 0.006329113924050633, 'yasu': 0.012658227848101266, 'yorokobi': 0.0, 'kowa': 0.06962025316455696, 'ikari': 0.0}
20200113 {'iya': 0.0, 'yasu': 0.0024271844660194173, 'yorokobi': 0.0, 'kowa': 0.05339805825242718, 'ikari': 0.0}
20200114 {'iya': 0.003105590062111801, 'yasu': 0.003105590062111801, 'yorokobi': 0.0, 'kowa': 0.052795031055900624, 'ikari': 0.0}
20200115 {'iya': 0.002164

    index       date         厭         安         喜         怖         怒
0       0 2020-01-06  0.000000  0.000000  0.000000  0.083333  0.033333
1       0 2020-01-07  0.000000  0.000000  0.000000  0.093023  0.000000
2       0 2020-01-08  0.000000  0.000000  0.000000  0.038462  0.000000
3       0 2020-01-09  0.000851  0.006809  0.000851  0.032340  0.000851
4       0 2020-01-11  0.002674  0.005348  0.000000  0.064171  0.000000
5       0 2020-01-12  0.006329  0.012658  0.000000  0.069620  0.000000
6       0 2020-01-13  0.000000  0.002427  0.000000  0.053398  0.000000
7       0 2020-01-14  0.003106  0.003106  0.000000  0.052795  0.000000
8       0 2020-01-15  0.002165  0.004329  0.000000  0.056277  0.000000
9       0 2020-01-16  0.002483  0.006703  0.000497  0.058590  0.001738
10      0 2020-01-17  0.003042  0.009886  0.002281  0.073764  0.001521
11      0 2020-01-18  0.001905  0.004762  0.000952  0.081905  0.001905
12      0 2020-01-19  0.006897  0.006897  0.000766  0.065134  0.001533
13    

20200131 {'iya': 0.009698817652496878, 'yasu': 0.013396694616994016, 'yorokobi': 0.003876283309275508, 'kowa': 0.06797281736055923, 'ikari': 0.006114471998313249}
処理時間 0:05:59.416917


## (B) 月単位のデータ保存
上記の「特定の感情のみに絞り込んだ解析」にて定義した関数を利用するコード。メイン関数の終盤で，（可視化でなく）感情推定の結果を pickle に保存している点のみ。

データ採取期間とファイル名の対応は下表のとおり：

|期間|ファイル名|
|-|-|
|1月6日〜1月31日|Jan_2020.pickle|
|2月1日〜2月29日|Feb_2020.pickle|
|3月1日〜3月31日|Mar_2020.pickle|
|4月1日〜4月30日|Apr_2020.pickle|
|5月1日〜5月31日|May_2020.pickle|
|6月1日〜6月30日|Jun_2020.pickle|
|7月1日〜7月31日|Jul_2020.pickle|
|8月1日〜8月31日|Aug_2020.pickle|

**(注意) 2020年1〜6月はツイート数（データ量）が多いため，それなりに計算時間を要する**

In [52]:
if __name__ == '__main__':
    # 開始時刻をメモ
    start_time = datetime.datetime.now()
    
    results = dict()

    # ML-Ask にて定義される感情の一覧
    # 可視化の際，日本語を用いるため ML-Ask の出力との対応表も準備する
    emotions = ['iya', 'yasu', 'yorokobi', 'kowa', 'ikari']
    j_emotions = {'iya':'厭', 'yasu':'安', 'yorokobi':'喜', 'kowa':'怖', 'ikari':'怒'}

    # データ収集期間の指定 & 保存するファイル名の指定
    dates = set_period(2020, 8, 1, 2020, 8, 31)
    save_filename = "Aug_2020.pickle"

    # 各月日の感情を推定 ＆ カウントアップ
    for d in dates:
        # ファイル名，特にパスは各自の環境に合わせて！
        # filename = "/home/muto/Dropbox/ExpSSE3/Tweets/{}_MERS.txt".format(d) # for Linux
        filename = r"C:\Users\yoshi\Dropbox\ExpSSE3\Tweets\{}_MERS.txt".format(d) # for Windows

        # 感情推定
        emotion_counter = emotion_estimation(filename, emotions)
        print(d, emotion_counter, file=sys.stderr)
        
        # データを取れていない日に対処するための措置
        if emotion_counter == []: continue

        # 月日 "d" の感情カウントを保存
        results[d] = emotion_counter
    
    # 終了時刻をメモ
    end_time = datetime.datetime.now()
    
    print("処理時間 {}".format(end_time - start_time), file=sys.stderr)
    
    # pandas データフレームに値を押し込む
    # 理由１：Plotly Express で処理するため
    # 理由２：感情を日本語で表現するため
    import pandas as pd
    df = pd.DataFrame(columns=['date']+emotions)
    
    #import matplotlib.pyplot as plt
    #from matplotlib.font_manager import FontProperties
    #fp = FontProperties(fname=r'/usr/share/fonts/truetype/fonts-japanese-gothic.ttf', size=16)

    for d in dates:
        dt = datetime.datetime(int(d[0:4]), int(d[4:6]), int(d[6:8]))
        try:
            df = df.append(pd.DataFrame([[dt, results[d]['iya'], results[d]['yasu'],
                                          results[d]['yorokobi'], results[d]['kowa'], results[d]['ikari']]],
                                         columns=['date']+emotions))
        except:
            pass # データを取れていない日に対処するための措置
    
    # 列名をローマ字表記から日本語表記へリネーム
    df = df.rename(columns=j_emotions)
    
    df = df.reset_index()
    
    #print(df)
    
    # 実行結果の保存
    import pickle
    with open(save_filename, "wb") as f:
        pickle.dump(df, f)

20200801 {'iya': 0.004410624272974021, 'yasu': 0.009305932531989143, 'yorokobi': 0.0016963939511438542, 'kowa': 0.01652772392400155, 'ikari': 0.004507561070182241}
20200802 {'iya': 0.005398854120758043, 'yasu': 0.009475539885412075, 'yorokobi': 0.0017078007933010136, 'kowa': 0.01972234464521816, 'ikari': 0.0052335830762450415}
20200803 {'iya': 0.004296133479868119, 'yasu': 0.009291637526226396, 'yorokobi': 0.0011489659306624038, 'kowa': 0.01713457887900889, 'ikari': 0.0034468977919872115}
20200804 {'iya': 0.005218761282699314, 'yasu': 0.009813897003315062, 'yorokobi': 0.0009518495421275479, 'kowa': 0.020710933140775264, 'ikari': 0.004627958118620146}
20200805 {'iya': 0.006726457399103139, 'yasu': 0.014798206278026907, 'yorokobi': 0.0017937219730941704, 'kowa': 0.026457399103139014, 'ikari': 0.004035874439461884}
20200806 {'iya': 0.005097148624779208, 'yasu': 0.009639162250820087, 'yorokobi': 0.0012112036336109008, 'kowa': 0.01625031541761292, 'ikari': 0.0029270754478930103}
20200807 {'

## (C) 2020年1〜8月の感情分布の可視化 (pickle から読み込み版)
上記(A)のコードから可視化機能のみ流用したバージョン。冒頭は(B)のコードで生成した解析結果を読み込んでいる。

In [10]:
import pickle
import pandas as pd

# 各月の感情推定の結果を読み込む
with open(r"month_results_json\Jan_2020.pickle", "rb") as f:
    df1 = pickle.load(f) 
with open(r"month_results_json\Feb_2020.pickle", "rb") as f:
    df2 = pickle.load(f) 
with open(r"month_results_json\Mar_2020.pickle", "rb") as f:
    df3 = pickle.load(f) 
with open(r"month_results_json\Apr_2020.pickle", "rb") as f:
    df4 = pickle.load(f) 
with open(r"month_results_json\May_2020.pickle", "rb") as f:
    df5 = pickle.load(f) 
with open(r"month_results_json\Jun_2020.pickle", "rb") as f:
    df6 = pickle.load(f) 
with open(r"month_results_json\Jul_2020.pickle", "rb") as f:
    df7 = pickle.load(f) 
with open(r"month_results_json\Aug_2020.pickle", "rb") as f:
    df8 = pickle.load(f) 

# １〜８月の結果を連結
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=0)
# df = df.drop("index", axis=1)
df = df.loc[:,['date','厭', '安','喜','怖', '怒']]
df = df.reset_index()
print(df)

# Plotly Express を用いた可視化
import plotly_express as px
df_melt = df.melt(id_vars='date', value_vars=j_emotions.values()) # value_vars=emotions)
fig = px.line(df_melt, x="date", y="value", color='variable',
              title="Emotions ....")
fig.show()

     index       date         厭         安         喜         怖         怒
0        0 2020-01-06  0.000000  0.000000  0.000000  0.083333  0.033333
1        1 2020-01-07  0.000000  0.000000  0.000000  0.093023  0.000000
2        2 2020-01-08  0.000000  0.000000  0.000000  0.038462  0.000000
3        3 2020-01-09  0.000851  0.006809  0.000851  0.032340  0.000851
4        4 2020-01-11  0.002674  0.005348  0.000000  0.064171  0.000000
..     ...        ...       ...       ...       ...       ...       ...
232     26 2020-08-27  0.004104  0.007744  0.001471  0.015101  0.003330
233     27 2020-08-28  0.006194  0.007213  0.001333  0.015680  0.007291
234     28 2020-08-29  0.004338  0.008386  0.001639  0.012724  0.004820
235     29 2020-08-30  0.005751  0.008848  0.001217  0.014046  0.004534
236     30 2020-08-31  0.005241  0.008221  0.001439  0.011921  0.002980

[237 rows x 7 columns]
